In [1]:
!nvidia-smi

Mon May 25 15:45:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.116.00   Driver Version: 418.116.00   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   39C    P0    53W / 300W |      0MiB / 16130MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A   43C    P0    54W / 300W |     10MiB / 16130MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
import os
import sys
sys.path.append('..')

import numpy as np
import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

Num GPUs Available:  4


In [3]:
tf.__version__

'2.1.0'

In [4]:
from bert.model import create_albert_model
from bert.losses import ECE, masked_sparse_categorical_crossentropy
from bert.optimization import WarmUp


In [5]:
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1E-4,
    decay_steps=10000,
    end_learning_rate=0.0)

lr_schedule = WarmUp(
    initial_learning_rate=1E-4,
    decay_schedule_fn=lr_schedule,
    warmup_steps=1000)

optimizer = tfa_optimizers.LAMB(
    learning_rate=lr_schedule,
    weight_decay_rate=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-6,
    exclude_from_weight_decay=['layer_norm', 'bias'])

# strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = create_albert_model(model_dimension=768,
                                transformer_dimension=768 * 4,
                                num_attention_heads=768 // 64,
                                num_transformer_layers=12,
                                vocab_size=24,
                                dropout_rate=0.,
                                max_relative_position=128)
    
    model.compile(
        loss=masked_sparse_categorical_crossentropy,
        metrics=[ECE],
        optimizer=optimizer,
        experimental_run_tf_function=True)    

    
model.summary()

TypeError: 'module' object is not callable

In [ ]:
import os
from bert.dataset import create_masked_input_dataset
data_path = '/gpfs/alpine/proj-shared/bie108/split_uniref100'

train_data_dir = os.path.join(data_path, 'train_uniref100_split')
train_data_files = [os.path.join(train_data_dir, f) for f in os.listdir(train_data_dir)]

valid_data_dir = os.path.join(data_path, 'dev_uniref50_split')
valid_data_files = [os.path.join(valid_data_dir, f) for f in os.listdir(valid_data_dir)]

with tf.device('/CPU:0'):
    training_data = create_masked_input_dataset(
        sequence_path=train_data_files,
        max_sequence_length=512,
        fix_sequence_length=True,
        batch_size=16)
    
    training_data = training_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)    

    valid_data = create_masked_input_dataset(
        sequence_path=valid_data_files,
        max_sequence_length=512,
        batch_size=16)

    valid_data = valid_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
print(next(iter(valid_data)))

In [ ]:
model.fit(training_data, steps_per_epoch=50, epochs=5,
          verbose=1, validation_data=valid_data, validation_steps=10)

In [ ]:
390 / 8

In [ ]:
552 / 12

In [ ]:
masked, true = next(iter(training_data))
predictions = model.predict(masked)

In [ ]:
masked[:, 0]